## **Introduction** 

This [Spotify song dataset](https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs/?select=spotify_songs.csv) contains over 30,000 songs from artists of different genres. Each song has specific attributes which help the user discern its genre. These attributes include `danceability`, `energy`, `loudness`, `key`, `mode`, `speechiness`, `acousticness`, `instrumentalness`, `liveness`, `valence`, and `tempo`.

The problem statement is to classify the songs (based on the above attributes) into six genres: `pop`, `rock`, `latin`, `EDM`, `rap`, and `R&B`.

```{}
```

## Importing the dataset

First, we import the libraries required to perform the initial data analysis. The dataset, hosted on [Kaggle](https://www.kaggle.com/), is imported and stored in the variable `spotify_songs`.


In [ ]:
#| echo: true 

# Import the libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request
from pathlib import Path
import os
import zipfile

# Downloading and opening the dataset

csv_path = Path('datasets/archive.zip') # store the dataset in a local folder
url = 'https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs/download?datasetVersionNumber=2' # url to download the dataset

if not csv_path.is_file(): # check if the dataset directory exists. 
  Path("datasets").mkdir(parents=True,exist_ok=True) # Create the directory
  urllib.request.urlretrieve(url, csv_path)
  with zipfile.ZipFile(csv_path) as spotify_file:
    spotify_file.extractall(path='datasets')
    
spotify_songs = pd.read_csv(Path('datasets/spotify_songs.csv')) # Store the dataset in a variable

## Analyzing the Data

Let's look at the non-null entries in the dataset using the `info()` method below.


In [ ]:
#| echo: true

spotify_songs.info()

The output shows that the columns `track_name`, `track_artist`, and `track_album_name` have 5 null elements (i.e., they only have 32828 elements). Let's look at the genres and their counts in the dataset.


In [ ]:
#| echo: true

spotify_songs['playlist_genre'].value_counts()

Let us now look at how different genres compare on the basis of each attribute. But first, we notice that the features have wildly different scales. For example, `loudness` varies from -46.448 to 1.275 whereas tempo ranges from 0 to 1 (see histogram below). Therefore, we must standardize these attributes using the `StandardScaler`. Furthermore, features such as `valence` and `instrumentalness` have very long tails. To convert these attributes to "gaussian" like features, we will use the `boxcox` class from `scipy`.


In [ ]:
#| echo: true

spotify_features = spotify_songs.loc[:,['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
spotify_features.hist(bins=50, figsize=(12, 8))
plt.show()

The following violin plot also describes the relationship between some of the attributes and genres. From the first violin plot, it is evident that `rap` music has a higher `danceability` score on average while the most danceable song belongs to the `edm` genre.


In [ ]:
#| echo: true

sns.violinplot(x=spotify_songs.playlist_genre,y=spotify_songs.danceability)
plt.grid()
plt.xlabel('Genre')
plt.ylabel('Danceability score')
plt.show()

We should also look for any correlations within the features themselves. Ideally, all our features (i.e., attributes) must be independent of each other. However, it may happen that two or more features are highly correlated within themselves. In such a scenario, we will need to drop one of the correlated attributes.


In [ ]:
#| echo: true

fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(spotify_features.corr(), annot=True, cmap='YlGnBu', vmin=-1, vmax=1, center=0, ax=ax)
plt.title('Linear Correlations between attributes')
plt.show()

From the correlation matrix, we can note that the attributes `energy` and `loudness` are highly correlated. Their correlation coefficient is 0.68. Thus, we will drop the `loudness` attribute.

## Preprocessing the data

We will first drop the `loudness` attribute since it is highly correlated with the `energy` attribute.


In [ ]:
#| echo: true

spotify_songs = spotify_songs.drop('loudness',axis=1)

The column `playlist_subgenre` has attributes such as dance pop and pop edm which are subsets of the parent column `playlist_genre`. Similarly,the `track_id`, `track_name`, `track_artist`, `track_popularity`, `track_album_id`, `track_album_name`, and `track_album_release_date` are not relevant. Thus, we drop all these columns. Furthermore, as we have seen before, there are null values in the `track_name`, `track_artist` , and `track_album_name` columns. Therefore, we need to drop the corresponding rows as well.


In [ ]:
#| echo: true

spotify_songs.dropna(subset=['track_name','track_artist','track_album_name'], inplace=True) # Drop null values from the data
spotify_songs = spotify_songs.loc[:,'playlist_genre':'tempo'] # drop unnecessary columns
spotify_songs = spotify_songs.drop('playlist_subgenre',axis=1) # drop unnecessary columns

We can verify that the null values have been dropped by calling the `.info()` method again.


In [ ]:
#| echo: true

spotify_songs.info()

Next, we will use `StandardScaler` transformer to standardize our attributes. Notice that the `spotify_features` dataframe in the following code does not include the `loudness` attribute.


In [ ]:
#| echo: true

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox

std_scaler = StandardScaler()

spotify_features = spotify_songs.loc[:,['danceability','energy','speechiness','acousticness','instrumentalness','liveness','valence','tempo']] # collecting all the important features in a dataframe
spotify_songs.loc[:,['danceability','energy','speechiness','acousticness','instrumentalness','liveness','valence','tempo']] = std_scaler.fit_transform(spotify_features)

# Using boxcox to scale heavy-tailed features

min_max_scaler = MinMaxScaler()
spotify_new_features = spotify_songs.loc[:,'speechiness':'liveness']
spotify_tail_features = min_max_scaler.fit_transform(spotify_new_features)
spotify_tail_features = pd.DataFrame(spotify_tail_features,columns=spotify_new_features.columns)
transformed_acousticness = boxcox(spotify_tail_features.acousticness,0.25) 
spotify_songs.acousticness = transformed_acousticness
transformed_liveness = boxcox(spotify_tail_features.acousticness,0.25)
spotify_songs.liveness = transformed_liveness
transformed_instrumentalness = boxcox(spotify_tail_features.instrumentalness,0.2) 
spotify_songs.instrumentalness = transformed_instrumentalness
transformed_speechiness = boxcox(spotify_tail_features.speechiness,0.2)
spotify_songs.speechiness = transformed_speechiness

Now, we can take a look at the feature distributions. As we can see, the attributes are on the same scale and many of them are centered around zero. Most of these distributions are Gaussian.


In [ ]:
#| echo: true

spotify_features = spotify_songs.loc[:,['danceability','energy','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
spotify_features.hist(figsize=(12, 8))
plt.show()

## Creating Training and Test Sets

Now, we will create training and test sets. We will do this by limiting the test set size to 20 % of the total data set. Since we need an equal distribution of our target labels (i.e., the playlist genres), we will need to stratify the data with respect to the playlist genres.


In [ ]:
#| echo: true

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(spotify_songs, test_size=0.2, stratify = spotify_songs.playlist_genre,random_state=42)

X_train = train_set.drop('playlist_genre',axis=1)
y_train = train_set['playlist_genre']
X_test = test_set.drop('playlist_genre',axis=1)
y_test = test_set['playlist_genre']

## Train the models

Since this is a [multiclass classification problem](https://scikit-learn.org/stable/modules/multiclass.html#:~:text=Multiclass%20classification%20is%20a%20classification,an%20apple%2C%20or%20a%20pear.), support vector classifiers do not scale very well. Therefore, we will look for other classifiers. First, we'll try logistic regression.


In [ ]:
#| echo: true

from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(max_iter=1000, random_state=42)
log_clf.fit(X_train, y_train); # Train the model

To measure the performance of the classifier, we will first import the `cross_val_score` function from scikit learn.


In [ ]:
#| echo: true

from sklearn.model_selection import cross_val_score

cross_val_score(log_clf,X_train,y_train,cv=3,scoring='accuracy')

For the 3 validation folds, Logistic Regression predicts the correct genres with 45 % accuracy on average. Let us compare this performance with a dummy classifier.


In [ ]:
#| echo: true


from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier()
dummy_clf.fit(X_train, y_train)

cross_val_score(dummy_clf, X_train, y_train, cv=3,scoring='accuracy')

The dummy classifier predicts the correct genre with 18 % accuracy. The Logistic Regression model is a little better than the dummy classifier for our data. Let us use the `DecisionTreeClassifier` next. We will use `RandomizedSeachCV` to select the optimum number of `max_feature`s in order to produce the best validation score.


In [ ]:
#| echo: true

from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

decision_tree_clf = DecisionTreeClassifier(random_state=42)
param_random = {'max_features':randint(low=2,high=20)}
random_search_tree = RandomizedSearchCV(decision_tree_clf,param_distributions=param_random,n_iter=10,cv=5,scoring='accuracy')
random_search_tree.fit(X_train, y_train)

cross_val_score(random_search_tree, X_train, y_train, cv=3,scoring='accuracy')

The `DecisionTreeClassifier` produces an average accuracy of 41 % on the validation folds. We will import the `RandomForestClassifier` , which aggregates a large number of decision trees together.


In [ ]:
#| echo: true

from sklearn.ensemble import RandomForestClassifier

random_forest_clf = RandomForestClassifier(n_jobs=-1,random_state=42)
param_random = [{'n_estimators':randint(low=100, high=500)},{'max_leaf_nodes':randint(low=5,high=20)}]
random_search_forest = RandomizedSearchCV(random_forest_clf,param_distributions=param_random,n_iter=10,cv=3,scoring='accuracy')
random_search_forest.fit(X_train,y_train)

cross_val_score(random_search_forest, X_train, y_train, cv=3,scoring='accuracy')

The `RandomForestClassifier` does better than the other models with an average accuracy of 53 % on the validation folds. Let us go one step further by using the `AdaBoostClassifier` with the `RandomForestClassifier` as the base estimator with a `learning_rate` of 0.5.


In [ ]:
#| echo: true

from sklearn.ensemble import AdaBoostClassifier

best_estimator = random_search_forest.best_estimator_
ada_boost = AdaBoostClassifier(best_estimator,n_estimators=5,learning_rate = 0.5, random_state=42)
ada_boost.fit(X_train, y_train)
cross_val_score(ada_boost, X_train, y_train, cv=3,scoring='accuracy')

The `AdaBoostClassifier` actually does slightly worse than the `RandomForestClassifier` on the validation folds. Therefore, let us use the `RandomForestClassifier` for the rest of our analysis. Firstly, let us measure the accuracy of the `RandomForestClassifier` on the test set.


In [ ]:
#| echo: true


random_search_forest.score(X_test, y_test) # print the accuracy score on the test set

This classifier has an accuracy score of 53.7 % on the test set.

## Error Analysis

The code below produces a Confusion Matrix for the `RandomForestClassifier`. As shown in the confusion matrix, many `edm` tracks (14 %) have been misclassified as `pop`. 16 % of the `latin` tracks have been misclassified as `pop` and `rap` each. Similarly, 17 % of `pop` tracks have been misclassified as `edm` and `r&b` each. 21 % of `r&b` tracks have been misclassified. These misclassifications have been caused due to similarities in the genres themselves. For instance, many `r&b` and `rap` songs have similarity `danceability` and `speechiness`. Further data collection is needed to differentiate the genres adequately.


In [ ]:
#| echo: true

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import ConfusionMatrixDisplay

y_preds = cross_val_predict(random_search_forest.best_estimator_,X_train, y_train, cv=3)

ConfusionMatrixDisplay.from_predictions(y_train,y_scores,normalize='true',values_format='.0%') # normalize predictions by row
plt.show()